In [ ]:
import anthropic
import json
import csv
import os
from datetime import datetime
from typing import List, Dict
import time

class AnthropicBatchCaller:
    def __init__(self, api_key: str, model: str, max_tokens: int):
        """
        Initialize the batch caller with API key, model, and max tokens.
        """
        self.client = anthropic.Anthropic(api_key=api_key)
        self.results = []
        self.prompt = ""
        self.model = model
        self.max_tokens = max_tokens
        
    def set_prompt(self):
        """Get prompt from user input"""
        while True:
            print("\n" + "="*50)
            print("PROMPT INPUT")
            print("="*50)
            print("Enter your prompt (press Enter when finished):")
            
            self.prompt = input().strip()
            
            if not self.prompt:
                print("Prompt cannot be empty. Please try again.")
                continue
            
            print(f"\nPrompt set: {len(self.prompt)} characters")
            print(f"Preview: {self.prompt[:100]}{'...' if len(self.prompt) > 100 else ''}")
            
            confirm = input("\nProceed with this prompt? (Y/n): ").strip().lower()
            
            if confirm in ['', 'y', 'yes']:
                break
            else:
                print("Let's try again...")
                continue
        
    def make_single_call(self, call_number: int) -> Dict:
        """Make a single API call and return the result"""
        try:
            print(f"Making call {call_number}/100...", end=" ", flush=True)
            
            response = self.client.messages.create(
                model=self.model,
                max_tokens=self.max_tokens,
                temperature=1.0,
                messages=[
                    {
                        "role": "user",
                        "content": self.prompt
                    }
                ]
            )
            
            result = {
                "call_number": call_number,
                "timestamp": datetime.now().isoformat(),
                "prompt": self.prompt,
                "response": response.content[0].text,
                "model": self.model,
                "temperature": 1.0,
                "max_tokens": self.max_tokens,
                "input_tokens": response.usage.input_tokens,
                "output_tokens": response.usage.output_tokens,
                "total_tokens": response.usage.input_tokens + response.usage.output_tokens,
                "status": "success"
            }
            
            print("✓ Success")
            return result
            
        except Exception as e:
            print(f"✗ Error: {str(e)}")
            return {
                "call_number": call_number,
                "timestamp": datetime.now().isoformat(),
                "prompt": self.prompt,
                "response": "",
                "model": self.model,
                "temperature": 1.0,
                "max_tokens": self.max_tokens,
                "input_tokens": 0,
                "output_tokens": 0,
                "total_tokens": 0,
                "status": "error",
                "error": str(e)
            }
    
    def make_batch_calls(self, num_calls: int = 100, delay: float = 1.0):
        """Make multiple API calls with optional delay between calls"""
        print(f"\nStarting batch of {num_calls} calls...")
        print(f"Delay between calls: {delay} seconds")
        
        # Initialize files for real-time updates
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = f"anthropic_batch_results_{timestamp}"
        
        # Set save directory
        save_dir = r"YOUR_DIRECTORY"
        
        json_filename = os.path.join(save_dir, f"{base_filename}.json")
        csv_filename = os.path.join(save_dir, f"{base_filename}.csv")
        
        # Create CSV file with headers
        fieldnames = [
            "call_number", "timestamp", "prompt", "response", "model", 
            "temperature", "max_tokens", "input_tokens", "output_tokens", 
            "total_tokens", "status", "error"
        ]
        
        with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
        
        # Create empty JSON file (as array start)
        with open(json_filename, 'w', encoding='utf-8') as f:
            f.write('[\n')
        
        print(f"Real-time files created:")
        print(f"  JSON: {json_filename}")
        print(f"  CSV: {csv_filename}")
        print("="*50)
        
        self.results = []
        
        for i in range(1, num_calls + 1):
            result = self.make_single_call(i)
            self.results.append(result)
            
            # Append to CSV after each call
            with open(csv_filename, 'a', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                row = {field: result.get(field, "") for field in fieldnames}
                writer.writerow(row)
            
            # Append to JSON after each call
            with open(json_filename, 'a', encoding='utf-8') as f:
                if i > 1:  # Add comma before all entries except the first
                    f.write(',\n')
                json.dump(result, f, indent=2, ensure_ascii=False)
            
            # Add delay between calls to avoid rate limiting
            if i < num_calls:
                time.sleep(delay)
        
        # Close the JSON array
        with open(json_filename, 'a', encoding='utf-8') as f:
            f.write('\n]')
        
        print("\n" + "="*50)
        print("BATCH COMPLETED")
        print("="*50)
        
        # Summary statistics
        successful_calls = sum(1 for r in self.results if r["status"] == "success")
        failed_calls = num_calls - successful_calls
        total_tokens = sum(r["total_tokens"] for r in self.results)
        
        print(f"Successful calls: {successful_calls}")
        print(f"Failed calls: {failed_calls}")
        print(f"Total tokens used: {total_tokens}")
        
        return json_filename, csv_filename
        
    def save_results(self, base_filename: str = None):
        """Save results to both JSON and CSV files"""
        if not self.results:
            print("No results to save!")
            return
        
        # Generate filename with timestamp if not provided
        if not base_filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            base_filename = f"anthropic_batch_results_{timestamp}"
        
        # Save as JSON
        json_filename = f"{base_filename}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, indent=2, ensure_ascii=False)
        
        # Save as CSV
        csv_filename = f"{base_filename}.csv"
        fieldnames = [
            "call_number", "timestamp", "prompt", "response", "model", 
            "temperature", "input_tokens", "output_tokens", "total_tokens", 
            "status", "error"
        ]
        
        with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            
            for result in self.results:
                # Ensure all fields are present
                row = {field: result.get(field, "") for field in fieldnames}
                writer.writerow(row)
        
        print(f"Results saved to:")
        print(f"  JSON: {json_filename}")
        print(f"  CSV: {csv_filename}")
        
        return json_filename, csv_filename

def get_user_inputs():
    """Get API key, model, and max tokens from user"""
    print("CONFIGURATION")
    print("="*50)
    
    # Get API key
    api_key = input("Enter your Anthropic API key: ").strip()
    
    # Get model choice
    print("\nSelect Model:")
    models = {
        "1": "claude-sonnet-4-20250514",
        "2": "claude-3-7-sonnet-20250219", 
        "3": "claude-opus-4-20250514"
    }
    
    print("1. Claude Sonnet 4 (Latest)")
    print("2. Claude 3.7 Sonnet")
    print("3. Claude Opus 4")
    
    while True:
        choice = input("Enter choice (1, 2, or 3): ").strip()
        if choice in models:
            selected_model = models[choice]
            break
        print("Please enter 1, 2, or 3")
    
    # Get max tokens
    print("\nMax Tokens (1-4000):")
    while True:
        max_tokens_input = input("Enter max tokens (or press Enter for 4000): ").strip()
        
        if not max_tokens_input:
            max_tokens = 4000
            break
        
        try:
            max_tokens = int(max_tokens_input)
            if max_tokens <= 0:
                print("Please enter a positive number")
                continue
            if max_tokens > 4000:
                print("Maximum allowed is 4000 tokens")
                continue
            break
        except ValueError:
            print("Please enter a valid number")
    
    print(f"\nConfiguration set:")
    print(f"  API Key: {'*' * (len(api_key) - 8) + api_key[-8:] if len(api_key) > 8 else '[Hidden]'}")
    print(f"  Model: {selected_model}")
    print(f"  Max Tokens: {max_tokens}")
    
    return api_key, selected_model, max_tokens

def main():
    """Main function to run the batch caller"""
    print("ANTHROPIC API BATCH CALLER")
    print("="*50)
    
    # Get user configuration
    api_key, model, max_tokens = get_user_inputs()
    
    # Initialize the caller
    try:
        caller = AnthropicBatchCaller(api_key=api_key, model=model, max_tokens=max_tokens)
        print("✓ API client initialized successfully")
    except Exception as e:
        print(f"✗ Error initializing API client: {e}")
        print("Make sure your API key is correct and you have the anthropic package installed")
        print("Install with: pip install anthropic")
        return
    
    # Get prompt from user
    caller.set_prompt()
    
    # Ready to start
    print(f"\nReady to make 100 API calls with your prompt.")
    print("Starting batch calls...")
    
    # Make the calls
    filenames = caller.make_batch_calls(num_calls=100, delay=1.0)
    
    print("\n" + "="*50)
    print("All done! Files have been updated in real-time.")
    print("You can now use the CSV file for semantic clustering or other analysis.")
    print(f"Final files: {filenames[0]}, {filenames[1]}")

if __name__ == "__main__":
    main()